# CHAPTER5 コンポーネントでUI部品を作る

## 5.21 コンポーネントとは

画面上のUIオブジェクト単位（ヘッダー、フッターリンク集...）ごとにVueのテンプレートと、JSをまとめたもの。
* このようにすることで、再利用性が高くなる他、機能の独立を確保することができるので、保守性が高まる。
* 他の言語で言うところの、クラス、オブジェクト指向と同じ概念のようなもの。

## 5.22 コンポーネントの定義方法
各コンポーネントはrootのVueインスタンスが定義される前に定義する必要がある。

コンポーネントはグローバルないしはローカルの範囲で登録することができ、呼び出し側ではReactのようなカスタムタグを使う.
* グローバルコンポーネント
    * コンポ定義側
        * `Vue.component('コンポ名', {オプション})`
        ```js
        Vue.component('my-component', {
            template: '<div>A custom component!</div>'
        });
        ```
    * コンポ使用側
        * `<コンポ名></コンポ名>`
        ```html
        <my-component></my-component>
        ```
    描画すると、
    ```html
    <div>A custom component!</div>
    ```
* ローカルコンポーネント
    * 特定のコンポーネントの`components`オプションに登録する。
    ```js
    var Child = {
        template: '<div>A custom component!</div>'
    }
    new Vue({
        el: ...,
        ...
        components: {
        // <my-component>はこの子コンポーネントを参照します
        'my-component': Child
        }
    })
    ```
* アイコンなどの汎用性の高いものはグローバルコンポーネント、特定のコンポーネントのみで使うものはローカルコンポーネントとして登録するのが良い。

### コンポーネントのオプション
rootのVueインスタンスのオプションと同じようなものが使える。
```js
Vue.component('my-component', {
    template: '<div>A custom component!</div>',
    data: function () { //NOTE: dataは関数で定義する
        return {
        status: 'Critical'
        }
    },
    methods: {
        ...
    },
    computed: {
        ...
    },
    watch: {
        ...
    }
    ...
})
```
#### rootのVueインスタンスとの違い
* `data`の定義方法
    * コンポーネントの`data`はrootのVueインスタンスの`data`と違い、必ず関数で定義する必要がある。
    * これは、コンポーネントが複数存在する場合、それぞれのコンポーネントが同じデータを参照してしまうことを防ぐため。
    ```js
    Vue.component('my-component', {
        template: '<div>A custom component!</div>',
        data: function () {
            return {
                status: 'Critical'
            }
        }
    })
    ```
* root要素は単一であること
    * コンポーネントの親要素は必ず1つ
    ```js
    Vue.component('my-component', {
        template: '<div>A custom component!</div><p>And another one!</p>' //NG: <div>と<p>が同じ階層にある
    })
    Vue.component('my-component', {
        template: '<div>A custom component!</div>' //OK
    })
    ```

### コンポーネント間通信
コンポーネント間でデータをやりとりする方法は、以下の3つがある。
* props
* イベント
* Vuex

### 親子コンポーネント
コンポーネントインスタンスの`template`で指定したコンポーネントが子コンポーネントとなる。
```js
Vue.component('child-component', {
    template: '<p>子コンポーネント</p>'
})
```
### 親から子
親コンポーネントで子コンポーネントを呼び出すとき、値の受け渡しはHTMLの属性として渡す。
```html
<child-component msg="親から子へ"></child-component>
```
```js
// 子コンポーネント
Vue.component('child-component', {
    template: '<p>{{ msg }}</p>',
    /**受け取りたい属性名を定義 */
    props: ['msg']
})
```

* `props`で受け渡ししたデータはリアクティブであるため、親側で更新すると子側も更新される。
    * したがって子側でデータを更新することはできない。
    

### propsのデータ型を指定する

TypeScriptのように、propsを受け取りに型指定をすることができる。引数型が間違っていると、Vueコンパイル時に警告を出すことができる。

```js
Vue.component('child-component', {
    template: '<p>{{ msg }}</p>',
    props: {
        msg: String //msg は String型指定
    }
})
```
`props`は型付けだけでなく、デフォルト値や必須項目を指定することができる。
* TypeScript の interfaceみたいだね。

### 子から親
子コンポーネントの状態に応じて、親コンポーネントから何か動作させたい場合、`カスタムイベント`と`$emit`を使う。

* 親コンポーネントのコード
  ```html
    <child-component @イベント名="親コンポーネントのメソッド名"></child-component>
　```
* 子コンポーネントのコード
  ```js
    this.$emit('イベント名', 引数)
  ```
> この時、イベント名はケバブケースで記述すること。でないと、親コンポーネントでイベントを受け取れない。


## 5.24 スロットを使ったコンポーネントのカスタマイズ
`スロット`とは、親コンポーネント側から子コンポーネント側にHTMLを渡すことができる機能。

### デフォルトスロット

カスタムタグの内側のHTMLは、スロットのデフォルトとして渡される。
```html
<child-component>
    <p>デフォルトスロット</p>
</child-component>
```
このとき、`child-component`側のテンプレートで`<slot></slot>`を記述すると、親コンポーネント側で指定したHTMLが表示される。
* 子コンポーネント
```html
<template>
    <div>
        <slot>スロットイン</slot>
    </div>
</template>
```

